## How to run ml-powered cloud filtering on Docker

### Download necessary files to start the colonyos server

wget https://raw.githubusercontent.com/colonyos/colonies/main/docker-compose.env  
wget https://raw.githubusercontent.com/colonyos/colonies/main/docker-compose.yml  

Then run:  
source docker-compose.env; docker-compose up  

### Now a colony server should start after downloading and installing all necessary components.  

Try:  
colonies colony ls  

It should return:  
NAME │ COLONYID  
dev  │ 4787a5071856a4acf702b2ffcea422e3237a679c681314113d86139461290cf4  

### Start by syncing a folder
colonies fs sync -l my_cloud_filtering -d ./my_cloud_filtering --yes  

#### Then check it out with  
colonies fs label ls  
and  
colonies fs ls -l /my_cloud_filtering  
To list files in the folder

### Let's create a function specification for colonies that we will send to colonies for processing

Copy the following to a .json file

In [ ]:
{
    "conditions": {
    	"executortype": "container-executor",
        "nodes": 1,
        "processespernode": 1,
	    "mem": "10Gi",
        "cpu": "1000m",
        "gpu": {
            "count": 0
        },
	"walltime": 1100
    },
    "maxexectime": 1000,
    "maxretries": 3,
    "funcname": "execute",
    "kwargs": {
        "cmd": "cd ",
        "docker-image": "henrikforsgren/openeo:new",
		"args": [
            "cfs/my_cloud_filtering/ ; python3 main.py"
		]
    },
	"env": {
		"coords": "{\"east\": 14.79187736312752, \"south\": 55.991257253340635, \"west\": 14.555719745816692, \"north\": 56.10331290101734}",
        "date": "2022-01-01",
		"data_source": "l2a"
	},
    "fs": {
        "mount": "/cfs",
        "dirs": [
            {
                "label": "/my_cloud_filtering",
                "dir": "/my_cloud_filtering",
                "keepfiles": false,
                "onconflicts": {
                    "onstart": {
                        "keeplocal": false
                    },
                    "onclose": {
                        "keeplocal": false 
                    }
                }
            },
            {
                "label": "/models",
                "dir": "/models",
                "keepfiles": false,
                "onconflicts": {
                    "onstart": {
                        "keeplocal": false
                    },
                    "onclose": {
                        "keeplocal": false 
                    }
                }
            },
            {
                "label": "/outputs",
                "dir": "/outputs",
                "keepfiles": false,
                "onconflicts": {
                    "onstart": {
                        "keeplocal": false
                    },
                    "onclose": {
                        "keeplocal": true 
                    }
                }
            }
        ]
    }
}


Important in the above .json file is the  
"executortype" which controls what type of colony executor gets to run the function.  
"cmd" the main command executed in the executor  
"args" which tells the executor the additional commands to run  
"env" contains the information the executed program in "args" will be able to use by collecting enviormental variables.  
"docker-image" will be the container started up by the docker-executor. Make sure you create or use a container that fulfills necessary requirements, like software enviorment and python packages

Finally, the "fs" describes how the colonies filesystem (fs) will act on start-up and closing of the process. The "my_cloud_filtering" and "models" both are just intended to load the same models and python code for each process. You can change the models and code to be executed by running " colonies fs sync -l my_cloud_filtering -d ./my_cloud_filtering --yes " again with updated code.  

Utilize the sync.sh file to sync ALL folders to colonies fs  
bash sync.sh  
or run  
colonies fs sync -l models -d ./models --yes  
colonies fs sync -l my_cloud_filtering -d ./my_cloud_filtering --yes  
colonies fs sync -l outputs -d ./outputs --yes  



Now when everything is setup, we can now start sending function requests with  
colonies function submit --spec get_cloud_free.json --follow

You can also use process_dates.sh to send multiple processes with a date range.  
./process_dates.sh 2022-01-01 2022-02-01
which is equivalent to doing colonies function submit --spec filtering.json and changing the "date" field to every single date from 2022-01-01 to 2022-02-01  

You can now see the processes being run and those who are in queue with  
colonies process ps  
and  
colonies process psw  
respectively 

When a image was found and determined to not be cloudy, it will be saved and uploaded to /outputs in the colony filesystem  
check them out with  
colonies fs ls -l /outputs  
and download them with  
colonies fs sync -l outputs -d ./outputs --yes  

You can now find the in your local /outputs folder  
ls /outputs